In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
import os
import time
import warnings
from typing import List
from datetime import datetime, timedelta

import json
import arrow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import (
    classification_report, 
    accuracy_score,
    f1_score,
    confusion_matrix, 
    roc_curve, 
    roc_auc_score, 
    RocCurveDisplay
)
from langchain.llms import OpenAI


In [4]:
warnings.filterwarnings("ignore")

sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

## Testbed for building language models

https://langchain.readthedocs.io/en/latest/getting_started/getting_started.html

In [ ]:
%set_env OPENAI_API_KEY=

In [13]:
llm = OpenAI(temperature=0.9, openai_api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
text = "Is it over?"
llm(text)